In [1]:
import sys
sys.path.append('E:/zlab/')
from loader import Loader
from xtrain import XModel, Bunch
# ----------------

import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tables as tb
import mxnet as mx
from mxnet import nd, autograd, gluon
import gluonbook as gb
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
from mxboard import SummaryWriter

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 载入数据

In [2]:
import tables as tb

h5 = tb.open_file('E:/xdata/X.h5')
#h5 = tb.open_file('./X.h5')

data = h5.root.cifar10

## 划分数据集

In [3]:
from sklearn.model_selection import train_test_split

X = data.trainX[:]
y = data.trainY[:]
X_train, X_unlabel, y_train, y_unlabel = train_test_split(
    X, y, test_size=0.98, random_state=42)

batch_size = 32
trainset = Loader(batch_size, X_train, y_train, shuffle=True, name='train')
testset = Loader(
    batch_size, data.testX, data.testY, shuffle=False, name='test')
unlabelset = Loader(
    batch_size,
    X_unlabel[:10000],
    y_unlabel[:10000],
    shuffle=False,
    name='agent')

# 模型的配置

In [8]:
class SemiModel(nn.HybridBlock):
    def __init__(self, features, **kwargs):
        super().__init__(**kwargs)
        self.features = features
        self.output = nn.Dense(10)
        
    def hybrid_forward(self, F, x):
        x = self.features(x)
        return self.output(x)

pretrain_net = model_zoo.vision.resnet50_v2(pretrained=False)
net = SemiModel(pretrain_net.features)

# 网络预设
_net = model_zoo.vision.resnet50_v2(pretrained=True)
net.features = _net.features
#net.features.add(nn.Flatten())
#net.features[-1].initialize(init.Xavier(magnitude=0.24))
net.output.initialize(init.Xavier(magnitude=0.24))

# 模型超参数设定

In [9]:
model = XModel(learning_rate = 0.1)
ctx = model.ctx
print('training on', ctx)
net.collect_params().reset_ctx(ctx)
net.hybridize()

training on gpu(0)


# 数据打包

In [10]:
dataset = Bunch(0.025, ctx, batch_size, trainset, unlabelset)

微调前的预测精度:  0.0973


In [11]:
y_pre = dataset.y_hats
dataset.agent(net)
y_back = dataset.y_hats

微调前的预测精度:  0.0973
微调后的预测精度:  0.0973


# 训练和调优

In [ ]:
logdir = 'D:/graph/'
sw = SummaryWriter(logdir, flush_secs=5)  # 可视化

epoch = 1
while epoch < 1000:
    print('微调有标签数据')
    print('~_~' * 25)
    sw.add_scalar('accuracy_curves',
                  {unlabelset.name + '_SSL': dataset.agency_acc}, epoch)
    net = model.train(net, dataset.train, testset, epochs=10, start=epoch)
    dataset.agent(net)
    print(unlabelset.name + '_SSL: ', dataset.agency_acc)
    epoch += 10
    if -1 in dataset.y_agent:
        dataset.y_agent = y_pre
        dataset.y_agent[dataset.y_hats != y_back] = -1 
    else:
        dataset.y_agent = np.ones_like(y_back)
        dataset.y_agent[dataset.y_hats != y_pre] = -1 
        
    #X_u_set = Loader(batch_size,dataset.unlabel.X,dataset.y_hats,shuffle=True,name='train_agent')
    #print('-_' * 50)
    #print('微调带有代理标签数据')
    #net = model.train(net, X_u_set, testset, epochs=1, start=epoch)
    dataset.agent(net)
    #epoch += 1